In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import log_loss, roc_auc_score
from deepctr.models import DeepFM,PNN
from deepctr.inputs import  SparseFeat, DenseFeat,get_feature_names
import math
import numpy as np
from keras.callbacks import EarlyStopping
import kerastuner
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.preprocessing.sequence import pad_sequences


In [19]:
data = pd.read_csv("./movielens_sample.txt")
sparse_features = ["movie_id", "user_id",
                   "gender", "age", "occupation", "zip"]
target = ['rating']

# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

In [5]:
data['age'].head()

0    2
1    1
2    2
3    1
4    5
Name: age, dtype: int64

In [6]:
def split(x):
    key_ans = x.split('|')
    for key in key_ans:
        if key not in key2index:
            # Notice : input value 0 is a special "padding",so we do not use 0 to encode valid feature for sequence input
            key2index[key] = len(key2index) + 1
    return list(map(lambda x: key2index[x], key_ans))

In [9]:
key2index = {}
genres_list = list(map(split, data['genres'].values))
genres_length = np.array(list(map(len, genres_list)))
max_len = max(genres_length)
# Notice : padding=`post`
genres_list = pad_sequences(genres_list, maxlen=max_len, padding='post', )

In [10]:
genres_list[0:5]

array([[1, 2, 0, 0, 0],
       [3, 4, 0, 0, 0],
       [2, 5, 0, 0, 0],
       [3, 6, 0, 0, 0],
       [1, 2, 0, 0, 0]], dtype=int32)

In [11]:
data['genres']=genres_list

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
user_id       200 non-null int64
movie_id      200 non-null int64
rating        200 non-null int64
timestamp     200 non-null int64
title         200 non-null object
genres        200 non-null int32
gender        200 non-null int64
age           200 non-null int64
occupation    200 non-null int64
zip           200 non-null int64
dtypes: int32(1), int64(8), object(1)
memory usage: 15.0+ KB


In [20]:
print(data['genres'].values[:5])
genres_list[:5]

['Comedy|Drama' 'Action|Thriller' 'Drama|Romance' 'Action|Adventure'
 'Comedy|Drama']


array([[1, 2, 0, 0, 0],
       [3, 4, 0, 0, 0],
       [2, 5, 0, 0, 0],
       [3, 6, 0, 0, 0],
       [1, 2, 0, 0, 0]], dtype=int32)

In [24]:
model_input = {name:data[name] for name in sparse_features}#
model_input["genres"] = genres_list